# Import

In [1]:
from functools import reduce
from pathlib import Path
import time
import datetime

import pandas as pd
import requests
from wmfdata import hive, spark
from wmfdata.utils import print_err, pd_display_all

# Parameters

In [2]:
# TSV file where metrics are or will be saved
FILENAME = "metrics/metrics.tsv"

# Metric month. The mediawiki_history snapshot must be from the metrics month or later.
last_month = datetime.date.today().replace(day=1) - datetime.timedelta(days=1)

METRICS_MONTH_TEXT = last_month.strftime("%Y-%m")
MEDIAWIKI_HISTORY_SNAPSHOT = last_month.strftime("%Y-%m")

# Preparation

In [13]:
# Convert our metrics month to all the formats we need and provide them in a dict
# so we can easily use them to format strings
metrics_month = METRICS_MONTH_TEXT
date_params = {
    "mediawiki_history_snapshot": MEDIAWIKI_HISTORY_SNAPSHOT,
    "metrics_month": str(metrics_month), 
    "metrics_month_first_day": str((datetime.date.today()- datetime.timedelta(days=31)).replace(day=1)),
    "metrics_month_last_day": str(last_month),
    "metrics_year": last_month.year,
    "metrics_cur_month" : last_month.month
}

# Load any previous results
try:
    old_metrics = (
        pd.read_csv(FILENAME, sep="\t", parse_dates = ["month"])
        .set_index("month")
    )
except FileNotFoundError:
    old_metrics = None
    
def prepare_query(filename):
    return (
        Path(filename)
        .read_text()
        .format(**date_params)
    )

In [15]:
date_params

{'mediawiki_history_snapshot': '2022-03',
 'metrics_month': '2022-03',
 'metrics_month_first_day': '2022-03-01',
 'metrics_month_last_day': '2022-03-31',
 'metrics_year': 2022,
 'metrics_cur_month': 3}

# Query metrics


In [16]:
queries = {
    "structured_data_used": {
        "file": "queries/structured_data_used.hql",
        "engine": "spark"
    },
    
    "wikidata_items": {
        "file": "queries/wikidata_items.hql",
        "engine": "spark"
    },
    "wikidata_items_being_reused": {
        "file": "queries/wikidata_items_being_reused.hql",
        "engine": "spark"
    }
       
}


for key, val in queries.items():
    query = prepare_query(val["file"])
    engine = val["engine"]
    print_err("Running {} on {}...".format(key, engine))
    
    if engine == "mariadb":
        result = mariadb.run(query)
    elif engine == "spark":
        result = spark.run(query)
        print(result)
    else:
        raise ValueError("Unknown engine specified.") 
    
    result = result.assign(month=lambda df: pd.to_datetime(df["month"]))
    val["result"] = result

Running structured_data_used on spark...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


        month  structured_data_used
0  2022-03-01             127991760


Running wikidata_items on spark...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


        month  wikidata_items
0  2022-03-01        96647383


Running wikidata_items_being_reused on spark...
PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


        month  wikidata_items_being_reused
0  2022-03-01                     20647318


In [17]:
#"num_commons_files_used_content_pages": {
#        "file": "queries/num_commons_files.hql",
#        "engine": "spark"
#    },



num_commons_files_used_content_pages = spark.run(""" 

WITH

    wiki_content_namespaces as (
        select
            dbname,
            namespace
        FROM
            wmf_raw.mediawiki_project_namespace_map
        WHERE
            namespace_is_content = 1
            AND snapshot = '2022-03'
    )

   
        SELECT
            '2022-03-01' AS month,
            COUNT(DISTINCT il_to) AS num_commons_files_used_content_pages
        FROM
            wmf_raw.mediawiki_imagelinks as m
            INNER JOIN wiki_content_namespaces AS ns
                ON ( ns.namespace = m.il_from_namespace)
                AND ns.dbname = m.wiki_db
        WHERE
            m.snapshot = '2022-03'

""")

PySpark executors will use /usr/lib/anaconda-wmf/bin/python3.


In [18]:
num_commons_files_used_content_pages

,month,num_commons_files_used_content_pages
0,2022-03-01,37488533


In [11]:
#num_commons_files_used_content_pages
test

,month,num_commons_files_used_content_pages
0,2022-02-01,37124441


# Combining and saving metrics

In [ ]:
# Assemble list of result dataframes
results = [val["result"] for _, val in queries.items()]

# Merge them all, assuming that the month is the only common column
new_metrics = reduce(lambda l, r: pd.merge(l, r, how="outer"), results)

# Set the month as an index so combine_first works properly
new_metrics = new_metrics.set_index("month").sort_index()

In [ ]:
if old_metrics is None:
    metrics = new_metrics
else:
    metrics = new_metrics.combine_first(old_metrics)
    
pd_display_all(metrics.tail(10))

In [ ]:
metrics.to_csv(FILENAME, sep="\t")